### This final code for the <span style="color:orange">'garfield'</span> function is designed to help facilitate prompt engineering by giving the engineer an accurate <span style="color:red">.csv</span> representation of the api response from the cat api, as well as counts to monitor token use and api calls.

In [7]:
import os
import json
from dotenv import load_dotenv
import requests
import pandas as pd
import numpy as np
import tiktoken
from datetime import datetime, timedelta

# set up the log for our cat api calls
cat_api_call_log = []

def garfield(save_path = "preened_cat_api_data.csv",
        limit = 10,
        verbose = False
            ):
    """
    Single function that calls The Cat API, flattens the JSON, and returns an easy-to-read .csv file for facilitated Prompt Engineering
    """

    global cat_api_call_log
    
    load_dotenv()
    cat_apikey = os.environ.get("cat_apikey")
    if not cat_apikey:
        raise ValueError("There is no 'cat_apikey' in your environment variables.")
    url = f"https://api.thecatapi.com/v1/images/search?size=med&mime_types=jpg&format=json&has_breeds=true&order=RANDOM&page=0&limit={limit}"
    headers = {
        'Content-Type': 'application/json',
        'x-api-key': cat_apikey
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        call_response = response.json()

        # track the call time and put it in the 'cat_api_call_log'
        now = datetime.now()
        cat_api_call_log.append(now)

        # number of api calls the past 24 hours
        cutoff = now - timedelta(days=1)
        recent_calls = [call for call in cat_api_call_log if call > cutoff]
        print(f"Number of API calls the last 24 hours: {len(recent_calls)}")

        
    except requests.exceptions.RequestException as e:
        print(f"Sorry. We could not execute your API call properly: {e}")
        return pd.DataFrame()  # empty fallback

    if verbose:
        print(json.dumps(call_response, indent=2))


    # running token count
    encoding = tiktoken.get_encoding("cl100k_base")
    token_count = len(encoding.encode(json.dumps(call_response)))

    print(f"Your api call consumed {token_count} tokens.")

    # flatten out the data
    data = []
    for item in call_response:
        for breed in item.get("breeds", []):
            data.append({
                "id": breed.get("id"), 
                "name": breed.get("name"), 
                "cfa_url": breed.get("cfa_url"),
                "vetstreet_url": breed.get("vetstreet_url"),
                "vcahospitals_url": breed.get("vcahospitals_url"), 
                "temperament": breed.get("temperament"), 
                "origin": breed.get("origin"), 
                "country_codes": breed.get("country_codes"),
                "country_code": breed.get("country_code"),
                "description": breed.get("description"),
                "life_span": breed.get("life_span"), 
                "indoor": breed.get("indoor"), 
                "lap": breed.get("lap"), 
                "alt_names": breed.get("alt_names"),
                "adaptability": breed.get("adaptability"),
                "affection_level": breed.get("affection_level"),
                "child_friendly": breed.get("child_friendly"),
                "dog_friendly": breed.get("dog_friendly"),
                "cat_friendly": breed.get("cat_friendly"),
                "energy_level": breed.get("energy_level"),
                "grooming": breed.get("grooming"),
                "health_issues": breed.get("health_issues"),
                "intelligence": breed.get("intelligence"),
                "shedding_level": breed.get("shedding_level"),
                "social_needs": breed.get("social_needs"),
                "stranger_friendly": breed.get("stranger_friendly"),
                "vocalisation": breed.get("vocalisation"),
                "experimental": breed.get("experimental"),
                "hairless": breed.get("hairless"),
                "natural": breed.get("natural"),
                "rare": breed.get("rare"),
                "rex": breed.get("rex"),
                "bidability": breed.get("bidability"),
                "suppressed_tail": breed.get("suppressed_tail"),
                "short_legs": breed.get("short_legs"),
                "wikipedia_url": breed.get("wikipedia_url"),
                "hypoallergenic": breed.get("hypoallergenic"),
                "reference_image_id": breed.get("reference_image_id"),
                "weight.imperial": breed.get("weight", {}).get("imperial"),
                "weight.metric": breed.get("weight", {}).get("metric"),
                "image_id": item.get("id"),
                "image_url": item.get("url")
            })

    api_df1 = pd.DataFrame(data)

    preferred_order = [
        "id", "name", "origin", "description", "alt_names", "temperament",
        "weight.metric", "weight.imperial", "intelligence", "child_friendly", "dog_friendly",
        "cat_friendly", "stranger_friendly", "energy_level","affection_level", "adaptability", 
        "indoor", "grooming", "hypoallergenic", "shedding_level", "life_span", "health_issues",
        "social_needs", "vocalisation", "lap", "natural", "experimental", "rare", "hairless", 
        "short_legs", "suppressed_tail", "bidability", "rex", "vcahospitals_url", "vetstreet_url",
        "cfa_url", "wikipedia_url", "image_url", "image_id", "reference_image_id", 
        "country_code", "country_codes"
    ]

    for column in preferred_order:
        if column not in api_df1.columns:
            api_df1[column] = np.nan

    api_df2 = api_df1[preferred_order]

    api_df2.to_csv(save_path, index=False)
    
    print(f"Your easy-to-read .csv file has been exported to: {save_path}")

    return api_df2

In [8]:
garfield()

Number of API calls the last 24 hours: 1
Your api call consumed 4666 tokens.
Your easy-to-read .csv file has been exported to: preened_cat_api_data.csv


,id,name,origin,description,alt_names,temperament,weight.metric,weight.imperial,intelligence,child_friendly,...,rex,vcahospitals_url,vetstreet_url,cfa_url,wikipedia_url,image_url,image_id,reference_image_id,country_code,country_codes
0,amau,Arabian Mau,United Arab Emirates,Arabian Mau cats are social and energetic. Due...,Alley cat,"Affectionate, Agile, Curious, Independent, Pla...",4 - 7,8 - 16,3,4,...,0,,None,None,https://en.wikipedia.org/wiki/Arabian_Mau,https://cdn2.thecatapi.com/images/zlpgGWqN7.jpg,zlpgGWqN7,k71ULYfRr,AE,AE
1,birm,Birman,France,"The Birman is a docile, quiet cat who loves pe...","Sacred Birman, Sacred Cat Of Burma","Affectionate, Active, Gentle, Social",3 - 7,6 - 15,3,4,...,0,https://vcahospitals.com/know-your-pet/cat-bre...,http://www.vetstreet.com/cats/birman,http://cfa.org/Breeds/BreedsAB/Birman.aspx,https://en.wikipedia.org/wiki/Birman,https://cdn2.thecatapi.com/images/9tZCXEYzy.jpg,9tZCXEYzy,HOrX5gwLS,FR,FR
2,bslo,British Longhair,United Kingdom,The British Longhair is a very laid-back relax...,,"Affectionate, Easy Going, Independent, Intelli...",4 - 8,8 - 18,5,4,...,0,None,None,None,https://en.wikipedia.org/wiki/British_Longhair,https://cdn2.thecatapi.com/images/re7uO34hz.jpg,re7uO34hz,7isAO4Cav,GB,GB
3,bsho,British Shorthair,United Kingdom,The British Shorthair is a very pleasant cat t...,"Highlander, Highland Straight, Britannica","Affectionate, Easy Going, Gentle, Loyal, Patie...",5 - 9,12 - 20,3,4,...,0,https://vcahospitals.com/know-your-pet/cat-bre...,http://www.vetstreet.com/cats/british-shorthair,http://cfa.org/Breeds/BreedsAB/BritishShorthai...,https://en.wikipedia.org/wiki/British_Shorthair,https://cdn2.thecatapi.com/images/_7U4xGLO_.jpg,_7U4xGLO_,s4wQfYoEk,GB,GB
4,cymr,Cymric,Canada,"The Cymric is a placid, sweet cat. They do not...",Spangle,"Gentle, Loyal, Intelligent, Playful",4 - 6,8 - 13,5,4,...,0,None,http://www.vetstreet.com/cats/cymric,None,https://en.wikipedia.org/wiki/Cymric_(cat),https://cdn2.thecatapi.com/images/OLP-tyC2i.jpg,OLP-tyC2i,3dbtapCWM,CA,CA
5,nebe,Nebelung,United States,"The Nebelung may have a reserved nature, but s...",Longhaired Russian Blue,"Gentle, Quiet, Shy, Playful",3 - 5,7 - 11,5,4,...,0,None,None,None,https://en.wikipedia.org/wiki/Nebelung,https://cdn2.thecatapi.com/images/FotU1pOJT.jpg,FotU1pOJT,OGTWqNNOt,US,US
6,rblu,Russian Blue,Russia,Russian Blues are very loving and reserved. Th...,"Archangel Blue, Archangel Cat","Active, Dependent, Easy Going, Gentle, Intelli...",2 - 5,5 - 11,3,3,...,0,https://vcahospitals.com/know-your-pet/cat-bre...,http://www.vetstreet.com/cats/russian-blue-neb...,http://cfa.org/Breeds/BreedsKthruR/RussianBlue...,https://en.wikipedia.org/wiki/Russian_Blue,https://cdn2.thecatapi.com/images/tHKLZkKZG.jpg,tHKLZkKZG,Rhj-JsTLP,RU,RU
7,sfol,Scottish Fold,United Kingdom,"The Scottish Fold is a sweet, charming breed. ...",Scot Fold,"Affectionate, Intelligent, Loyal, Playful, Soc...",2 - 5,5 - 11,3,4,...,0,https://vcahospitals.com/know-your-pet/cat-bre...,http://www.vetstreet.com/cats/scottish-fold-hi...,http://cfa.org/Breeds/BreedsSthruT/ScottishFol...,https://en.wikipedia.org/wiki/Scottish_Fold,https://cdn2.thecatapi.com/images/_GVSjXZ4i.jpg,_GVSjXZ4i,o9t0LDcsa,GB,GB
8,sphy,Sphynx,Canada,"The Sphynx is an intelligent, inquisitive, ext...","Canadian Hairless, Canadian Sphynx","Loyal, Inquisitive, Friendly, Quiet, Gentle",3 - 5,6 - 12,5,4,...,0,https://vcahospitals.com/know-your-pet/cat-bre...,http://www.vetstreet.com/cats/sphynx,http://cfa.org/Breeds/BreedsSthruT/Sphynx.aspx,https://en.wikipedia.org/wiki/Sphynx_(cat),https://cdn2.thecatapi.com/images/bSu2exlkB.jpg,bSu2exlkB,BDb8ZXb1v,CA,CA
9,tvan,Turkish Van,Turkey,"While the Turkish Van loves to jump and climb,...","Turkish Cat, Swimming cat","Agile, Intelligent, Loyal, Playful, Energetic",3 - 9,7 - 20,5,4,...,0,https://vcahospitals.com/know-your-pet/cat-bre...,http://www.vetstreet.com/cats/turkish-van,http://cfa.org/Breeds/BreedsSthruT/TurkishVan....,https://en.wikipedia.org/wiki/Turkis